In [26]:
import settings
import mysql.connector
import pandas as pd
import time
import itertools
import math
import credentials
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
#%matplotlib inline
import plotly.express as px
import datetime
from IPython.display import clear_output

import plotly.offline as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
# py.init_notebook_mode()
from wordcloud import WordCloud, STOPWORDS
import re
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import sqlite3
import country_converter as coco
import numpy as np
import collections
import warnings
import tweepy
from nltk.corpus import stopwords
stop = stopwords.words('english')
# nltk.download('stopwords')

In [27]:
conn1 = sqlite3.connect('twitter.db')
print("Opened the database successfully")
conn = conn1.cursor()


timenow = (datetime.datetime.utcnow() - datetime.timedelta(hours=0, minutes=10)).strftime('%Y-%m-%d %H:%M:%S')

# query = "SELECT * FROM {} WHERE created_at <= '{}' " .format(settings.TABLE_NAME, timenow)
query="select * from Facebook"
df = pd.read_sql(query, con=conn1)
df['created_at'] = pd.to_datetime(df['created_at'])

Opened the database successfully


In [4]:
print(df['user_location'])

0        GBR
1        IND
2        GBR
3        IND
4       None
        ... 
2189     GBR
2190     ITA
2191     USA
2192     IND
2193     HTI
Name: user_location, Length: 2194, dtype: object


In [52]:
normal_names = df["user_location"].dropna().tolist()
# normal_names = ['Philippines ' if x=='Luzon ' else x for x in normal_names]
# normal_names = ['Iran ' if x=='ایران ' else x for x in normal_names] 
# iso2_codes = coco.convert(names=normal_names, to='ISO3')
# warnings.filterwarnings("ignore")


counter=collections.Counter(normal_names)
df1 = pd.DataFrame.from_dict(counter, orient='index').reset_index()
df1 = df1.rename(columns={'index':'CODE', 0:'COUNT'})
country=list()
for i in df1['CODE']:
    country.append(coco.convert(names = i, to = 'name_short'))    
df1['COUNTRY']=country


In [53]:
print(df1.head())

  CODE  COUNT         COUNTRY
0  USA     25   United States
1  ITA     38           Italy
2  AUT      1         Austria
3  GBR     10  United Kingdom
4  AUS      1       Australia


In [54]:
# conn1 = sqlite3.connect('twitter.db')
# print("Opened the database successfully")
# conn = conn1.cursor()


# timenow = (datetime.datetime.utcnow() - datetime.timedelta(hours=0, minutes=10)).strftime('%Y-%m-%d %H:%M:%S')

# query = "SELECT * FROM {} WHERE created_at <= '{}' " .format(settings.TABLE_NAME, timenow)
# df = pd.read_sql(query, con=conn1)
# df['created_at'] = pd.to_datetime(df['created_at'])


# '''
# Country frequency counting and conversion
# '''


# normal_names = df["user_location"].dropna().tolist()
# normal_names = ['Philippines ' if x=='Luzon ' else x for x in normal_names]
# normal_names = ['Iran ' if x=='ایران ' else x for x in normal_names] 
# iso2_codes = coco.convert(names=normal_names, to='ISO3')
# warnings.filterwarnings("ignore")


# counter=collections.Counter(iso2_codes)
# df1 = pd.DataFrame.from_dict(counter, orient='index').reset_index()
# df1 = df1.rename(columns={'index':'CODE', 0:'COUNT'})
# country=list()
# for i in df1['CODE']:
#     country.append(coco.convert(names = i, to = 'name_short'))    
# df1['COUNTRY']=country


# Clean and transform data to enable time series
result = df.groupby([pd.Grouper(key='created_at', freq='2s'), 'polarity']).count().unstack(fill_value=0).stack().reset_index()
result = result.rename(columns={"id_str": "Num of '{}' mentions".format(settings.TRACK_WORDS[0]), "created_at":"Time in UTC"})  
time_series = result["Time in UTC"][result['polarity']==0].reset_index(drop=True)

#NER frequency counting
li=list()
for i in df["named_ent"]:    
    te=i.split(",")    
    li.extend(te)

li = list(filter(None, li))
fdist = FreqDist(li)
fd = pd.DataFrame(fdist.most_common(10), columns = ["Word","Frequency"]).drop([0]).reindex()

# Likes count
likes=df['favorite_count']


#User frequency count
user=list()
for i in df['users_list']:
    a=i.split(",")
    user.extend(a)
    
user = list(filter(None, user))

fdist1 = FreqDist(user)
fd1 = pd.DataFrame(fdist1.most_common(10), columns = ["Word","Frequency"]).drop([0]).reindex()

val_list=list()
val_list.append(df['polarity'].value_counts()[0])
val_list.append(df['polarity'].value_counts()[1])
val_list.append(df['polarity'].value_counts()[-1])


# time.sleep(60)

In [55]:
print(df1)

   CODE  COUNT                COUNTRY
0   USA     25          United States
1   ITA     38                  Italy
2   AUT      1                Austria
3   GBR     10         United Kingdom
4   AUS      1              Australia
5   PHL      1            Philippines
6   JOR      2                 Jordan
7   ARE      3   United Arab Emirates
8   CAN      5                 Canada
9   FRA      3                 France
10  IRL      2                Ireland
11  CHL      3                  Chile
12  NPL      1                  Nepal
13  DEU      4                Germany
14  LAO      1                   Laos
15  BGD      1             Bangladesh
16  ZAF      1           South Africa
17  NGA      5                Nigeria
18  IND      3                  India
19  BEL      1                Belgium
20  FSM      1  Micronesia, Fed. Sts.
21  BRA      1                 Brazil
22  PAK      2               Pakistan
23  JAM      1                Jamaica


In [47]:
# df.loc[df['user_location'] == "ایران "]

def get_location(temp_location):
    geolocator = Nominatim(user_agent="myGeocoder")         # Initializing geolocator object for getting address
    try:
        location = geolocator.geocode(temp_location)
        # print(location.raw['address']['country'])                
        location_list = location.raw['display_name'].split(",")
        user_location1=location_list[len(location_list)-1].strip()               # Extracting only country name
        translator = google_translator()  
        user_location = translator.translate(user_location1, lang_tgt='en')         #Translate the location to english
    except Exception as e:
        user_location = None

    return user_location


print(get_location('Éire / Ireland '))

Iran 


In [56]:


# fig = make_subplots(
#         rows=3, cols=2,
#         column_widths=[1, 0.4],
#         row_heights=[0.6, 0.6, 0.5],
#         specs=[[{"type": "choropleth" , "rowspan": 2}      , {"type": "bar"}],               
#                [None                             , {"type": "bar"}],          
#                [{"type": "scatter"}, {"type": "bar"}],
# #                [{"type": "choropleth"}      , {"type": "bar"}]
# #                [            {"type": "bar"},       None                   ]
#               ])

fig = make_subplots(
        rows=3, cols=1,
        column_widths=[1],
        row_heights=[1, 1, 1],
        specs=[[{"type": "choropleth" , "rowspan": 2}],              
               [None],              
               [{"type": "scatter"}]])
             

'''
Plot the Line Chart
'''
fig.add_trace(go.Scatter(
    x=time_series,
    y=result["Num of '{}' mentions".format(settings.TRACK_WORDS[0])][result['polarity']==0].reset_index(drop=True),
    name="Neural",
    opacity=0.8), row=3, col=1)   
fig.add_trace(go.Scatter(
    x=time_series,
    y=result["Num of '{}' mentions".format(settings.TRACK_WORDS[0])][result['polarity']==-1].reset_index(drop=True),
    name="Negative",
    opacity=0.8), row=3, col=1)
fig.add_trace(go.Scatter(
    x=time_series,
    y=result["Num of '{}' mentions".format(settings.TRACK_WORDS[0])][result['polarity']==1].reset_index(drop=True),
    name="Positive",
    opacity=0.8), row=3, col=1)

'''
Plot the Bar Chart
'''
# content = ' '.join(df["text"])
# content = re.sub(r"http\S+", "", content)
# content = content.replace('RT ', ' ').replace('&amp;', 'and')
# content = re.sub('[^A-Za-z0-9]+', ' ', content)
# content = content.lower()

# tokenized_word = word_tokenize(content)
# stop_words=set(stopwords.words("english"))
# filtered_sent=[]
# for w in tokenized_word:
#     if w not in stop_words:
#         filtered_sent.append(w)


'''
Plot the Geo-Distribution
'''
# is_in_US=[]
# geo = df[['user_location']]
# df = df.fillna(" ")
# for x in df['user_location']:
#     check = False
#     for s in STATES:
#         if s in x:
#             is_in_US.append(STATE_DICT[s] if s in STATE_DICT else s)
#             check = True
#             break
#     if not check:
#         is_in_US.append(None)

# geo_dist = pd.DataFrame(is_in_US, columns=['State']).dropna().reset_index()
# geo_dist = geo_dist.groupby('State').count().rename(columns={"index": "Number"}) \
#         .sort_values(by=['Number'], ascending=False).reset_index()
# geo_dist["Log Num"] = geo_dist["Number"].apply(lambda x: math.log(x, 2))


# geo_dist['Full State Name'] = geo_dist['State'].apply(lambda x: INV_STATE_DICT[x])
# geo_dist['text'] = geo_dist['Full State Name'] + '<br>' + 'Num: ' + geo_dist['Number'].astype(str)
# fig.add_trace(go.Choropleth(
#     locations=geo_dist['State'], # Spatial coordinates
#     z = geo_dist['Log Num'].astype(float), # Data to be color-coded
#     locationmode = 'USA-states', # set of locations match entries in `locations`
#     colorscale = "Blues",
#     text=geo_dist['text'], # hover text
#     showscale=False,
#     geo = 'geo'
#     ),
#     row=1, col=2)



fig.add_trace(go.Choropleth(
    locations = df1['CODE'],
    z = df1['COUNT'],
    text = df1['COUNTRY'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '$',
    colorbar_title = 'GDP<br>Billions US$',
), row=1, col=1)


fig.update_layout(
    title_text= "Real-time tracking '{}' mentions on Twitter {} UTC".format(settings.TRACK_WORDS[0] ,datetime.datetime.utcnow().strftime('%m-%d %H:%M')),
    geo = dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),   
#     template="plotly_dark",
#     margin=dict(r=20, t=50, b=50, l=20),
    annotations=
#     [
#         go.layout.Annotation(
#             text="Source: Twitter",
#             showarrow=False,
#             xref="paper",
#             yref="paper",
#             x=0,
#             y=0)
#     ],
    [dict(
        x=1,
        y=1,
        xref='paper',
        yref='paper',
        showarrow = False
    )]
)

fig.show()


In [6]:
fig = make_subplots(
        rows=2, cols=2,
        column_widths=[1, 1],
        row_heights=[0.4, 0.6],
        specs=[[  {"type": "bar"}   , {"type": "bar"}] ,          
               [ {"type": "pie"} , {"type": "bar"}]
#               [None, None],
              ])
               

    
'''
NER frequency plotting
'''

# Plot Bar chart   
fig.add_trace(go.Bar(x=fd["Word"], y=fd["Frequency"], name="Freq Dist"), row=1, col=1)
# 59, 89, 152
fig.update_traces(marker_color='rgb(59, 89, 152)', marker_line_color='rgb(8,48,107)', \
        marker_line_width=0.5, opacity=0.7, row=1, col=1)
# fig.update_layout( xaxis = dict(tickfont = dict(size=9)))


'''
User frequency count and plotting
'''

# Plot Bar chart   
fig.add_trace(go.Bar(x=fd1["Word"], y=fd1["Frequency"], name="Freq Dist"), row=1, col=2)
# 59, 89, 152

fig.update_traces(marker_color='rgb(59, 89, 152)', marker_line_color='rgb(8,48,107)', \
        marker_line_width=0.5, opacity=0.7, row=1, col=2)
# fig.update_layout( xaxis = dict(tickfont = dict(size=9)))

labels=[0,1,-1]
# This dataframe has 244 lines, but 4 distinct values for `day`

fig.add_trace(go.Pie(labels=labels, values=val_list), row=2, col=1)

# for i in fig['layout']['annotations']:
#     i['font'] = dict(size=10,color='#ff0000')

# fig.for_each_xaxis(lambda axis: axis.font.update(font=dict(size=10)))
fig.show()

In [19]:

date_since = "2021-03-01"

# authorization of consumer key and consumer secret 
auth = tweepy.OAuthHandler(credentials.consumer_key, credentials.consumer_secret) 

# set access to user's access key and access secret 
auth.set_access_token(credentials.access_token, credentials.access_token_secret) 

# calling the api 
api = tweepy.API(auth) 

# the ID of the status 
query="(facebook) -facebook.com min_faves:500"

# fetching the status 
status = api.search(query, lang="en", since=date_since, count=1000) 

like=dict()
for i in status:
    like[i.user.screen_name]=i.favorite_count

sort_orders = sorted(like.items(), key=lambda x: x[1], reverse=True)

likes=list()
usernames=list()
j=0
for i in sort_orders:
    if j<10:
        likes.append(i[1])
        usernames.append(i[0])
        j=j+1

fig = go.Figure([go.Bar(x=usernames, y=likes)])
fig.show()

In [38]:
# print(df['users_list'].head(20))
user=list()
# print(df['users_list'].head(20))
for i in df['users_list'][:20]:
    a=i.split(",")
    for j in a:
        user.append(j)
print(user)
user = filter(None, user)
counter=collections.Counter(user)
print(counter)

['', 'tedcruz', 'tedcruz', 'StarFMBaguio', '', 'tedcruz', '', '', 'BritneyDoll5', '', 'ironheadedge58', 'alex371', 'tweettruth2me', 'TexanBeauty86', 'queenalmamarie', 'DogsDre', 'DonaldJTrumpJr', 'tedcruz', 'BreesAnna', 'Intangiblejourneys', 'Martiansrepublics', 'Quantumdeviations', 'tedcruz', 'ghpex_com', '', 'bbrightvc', '', '']
Counter({'tedcruz': 5, 'StarFMBaguio': 1, 'BritneyDoll5': 1, 'ironheadedge58': 1, 'alex371': 1, 'tweettruth2me': 1, 'TexanBeauty86': 1, 'queenalmamarie': 1, 'DogsDre': 1, 'DonaldJTrumpJr': 1, 'BreesAnna': 1, 'Intangiblejourneys': 1, 'Martiansrepublics': 1, 'Quantumdeviations': 1, 'ghpex_com': 1, 'bbrightvc': 1})


In [12]:
print(df["named_ent"])
li=list()
for i in df["named_ent"]:    
    te=i.split(",")    
    if len(te) > 0:
        for j in te:
            li.append(j)
    else:
        li.append(j)
li = list(filter(None, li))
print(li)

0                           
1                           
2           Mazi Nnamdi Kanu
3                           
4       dog Benji,Mottingham
               ...          
259    a Professional Styler
260                         
261     Newsmax,Mike Lindell
262                    House
263                         
Name: named_ent, Length: 264, dtype: object
['Mazi Nnamdi Kanu', 'dog Benji', 'Mottingham', 'Newsmax', 'Mike Lindell', 'Gizmo', 'Myanmar', 'Instaspa', 'Mike Lindell', 'Tw', 'GRAPHIC SPECIAL PLEA Shadow', 'HORRIFIC LEG INJURY', 'Radio Biafra App', 'Mark Nzuzukerberge', 'India', 'Mike Lindell', 'Brew Co Bucks', 'WellingtonMain Gym  ', 'Twitter', 'FacebookArrested', 'House', 'US House', 'Rep Greene', 'HallColdwell Banker CandC Prope', 'House', 'Manhattan', 'the US Embassy', 'Asmara', 'Twitter', 'Insta', 'Newsmax', 'Mike Lindell', 'ShadowPledge', 'Tumblr', 'Instagram to Yahoo', 'Elon Musk', 'Mark Zuckerberg', 'Katie', 'Mike Lindell', 'MB First Nations', 'AMC', 'Subs', 'SNS', '

In [11]:
content = ' '.join(df["named_ent"])
content = re.sub(r"http\S+", "", content)
content = content.replace('RT ', ' ').replace('&amp;', 'and')
content = re.sub('[^A-Za-z0-9]+', ' ', content)
content = content.lower()

tokenized_word = word_tokenize(content)
stop_words=set(stopwords.words("english"))
filtered_sent=[]
for w in tokenized_word:
    if w not in stop_words:
        filtered_sent.append(w)
fdist = FreqDist(filtered_sent)
fd = pd.DataFrame(fdist.most_common(10), columns = ["Word","Frequency"]).drop([0]).reindex()

# Plot Bar chart   
fig.add_trace(go.Bar(x=fd["Word"], y=fd["Frequency"], name="Freq Dist"), row=2, col=2)
# 59, 89, 152
fig.update_traces(marker_color='rgb(59, 89, 152)', marker_line_color='rgb(8,48,107)', \
        marker_line_width=0.5, opacity=0.7, row=2, col=2)


3825

In [18]:
fdist = FreqDist(li)
# print(fdist)

# fdist = FreqDist(filtered_sent)
fd = pd.DataFrame(fdist.most_common(10), columns = ["Word","Frequency"]).drop([0]).reindex()
# print(fd)
# # Plot Bar chart   
fig.add_trace(go.Bar(x=fd["Word"], y=fd["Frequency"], name="Freq Dist"), row=2, col=2)
# 59, 89, 152
fig.update_traces(marker_color='rgb(59, 89, 152)', marker_line_color='rgb(8,48,107)', \
        marker_line_width=0.5, opacity=0.7, row=2, col=2)


AttributeError: 'Figure' object has no attribute 'add_trace'

In [66]:
import country_converter as coco
import pandas as np
import numpy as np
import sqlite3
import collections
import plotly.graph_objects as go

dataframe = pd.read_sql("""SELECT user_location FROM FACEBOOK """, con=conn1)


conn1 = sqlite3.connect('twitter.db')
print("Opened the database successfully")

conn = conn1.cursor()

query = "SELECT * from  {}" .format(settings.TABLE_NAME)
df = pd.read_sql(query, con=conn1)
normal_names = df["user_location"].tolist()
iso2_codes = coco.convert(names=normal_names, to='ISO3')
# print(iso2_codes)
counter=collections.Counter(iso2_codes)
# print(counter)
df1 = pd.DataFrame.from_dict(counter, orient='index').reset_index()
df1 = df1.rename(columns={'index':'CODE', 0:'COUNT'})
# df1['COUNTRY'] = cc.convert(names = df1['CODE'], to = 'name_short')
country=list()
for i in df1['CODE']:
    country.append(coco.convert(names = i, to = 'name_short'))
    
df1['COUNTRY']=country
# print(df1)


fig = go.Figure(data=go.Choropleth(
    locations = df1['CODE'],
    z = df1['COUNT'],
    text = df1['COUNTRY'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '$',
    colorbar_title = 'GDP<br>Billions US$',
))

fig.update_layout(
    title_text='2014 Global GDP',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.9,
        y=0.7,
        xref='paper',
        yref='paper',
#         text='Source: <a href="https://www.cia.gov/library/publications/the-world-factbook/fields/2195.html">\
#             CIA World Factbook</a>',
        showarrow = False
    )]
)

fig.show()

Luzon  not found in regex
Africa  not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex


Opened the database successfully


None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
C:\ProgramData\Anaconda3\lib\site-packages\country_converter\country_converter.py:578: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\ProgramData\Anaconda3\lib\site-packages\country_converter\country_converter.py:578: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\ProgramData\Anaconda3\lib\site-packages\country_converter\country_converter.py:578: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\ProgramData\Anaconda3\lib\site-packages\

In [11]:
import country_converter as coco
cc = coco.CountryConverter()

some_names = ['United Rep. of Tanzania', 'Cape Verde', 'Burma',
              'Iran (Islamic Republic of)', 'Korea, Republic of',
              "Dem. People's Rep. of Korea", "USA"]

standard_names = cc.convert(names = some_names, to = 'name_short')
UNmembership = cc.convert(names = some_names, to = 'UNmember')
print(type(standard_names))
print(UNmembership)

<class 'list'>
[1961, 1975, 1948, 1945, 1991, 1991, 1945]


C:\ProgramData\Anaconda3\lib\site-packages\country_converter\country_converter.py:578: FutureWarning:

The default value of regex will change from True to False in a future version.

